In [1]:
import yfinance as yf
import pandas as pd
import plotly.express as px
import numpy as np

In [2]:
# connect to MetaTrader5 as mt5

msft = yf.Ticker("MSFT")
hist = msft.history(period="5y")

In [8]:

start_pos = 0
num_bars = 1000

fsma_period = 10
ssma_period = 100

In [3]:
hist.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2017-11-14 00:00:00-05:00,78.460744,79.024533,77.972130,78.977554,18801300,0.00,0
2017-11-15 00:00:00-05:00,78.826478,79.034240,78.089871,78.363739,19383100,0.42,0
2017-11-16 00:00:00-05:00,78.477052,78.779250,78.325957,78.571487,20962800,0.00,0
2017-11-17 00:00:00-05:00,78.495933,78.495933,77.664884,77.815987,22079000,0.00,0
2017-11-20 00:00:00-05:00,77.815993,77.995419,77.674337,77.938759,16315000,0.00,0


In [20]:
# Requesting historical data

df = pd.DataFrame(hist)[[ 'Open', 'High', 'Low', 'Close']]
#df['Date'] = pd.to_datetime(df['Date'], unit='s')

df['fast_sma'] = df['Close'].rolling(fsma_period).mean()
df['slow_sma'] = df['Close'].rolling(ssma_period).mean()

# finding crossovers
df['prev_fast_sma'] = df['fast_sma'].shift(1)

df['time'] = df.index

df.dropna(inplace=True)
df

,Open,High,Low,Close,fast_sma,slow_sma,prev_fast_sma,time
Date,,,,,,,,
2018-04-10 00:00:00-04:00,87.660103,88.504539,86.948499,88.125015,86.051861,84.313705,86.137251,2018-04-10 00:00:00-04:00
2018-04-11 00:00:00-04:00,87.299533,88.514002,86.796668,87.157211,86.278625,84.395502,86.051861,2018-04-11 00:00:00-04:00
2018-04-12 00:00:00-04:00,87.698020,89.339455,87.698020,88.789146,86.676173,84.499756,86.278625,2018-04-12 00:00:00-04:00
2018-04-13 00:00:00-04:00,89.235108,89.358450,87.707531,88.314766,86.847907,84.597189,86.676173,2018-04-13 00:00:00-04:00
2018-04-16 00:00:00-04:00,89.254074,89.813873,88.637350,89.348953,87.383982,84.712518,86.847907,2018-04-16 00:00:00-04:00
...,...,...,...,...,...,...,...,...
2022-11-08 00:00:00-05:00,228.699997,231.649994,225.839996,228.869995,226.672000,255.139076,228.851001,2022-11-08 00:00:00-05:00
2022-11-09 00:00:00-05:00,227.369995,228.630005,224.330002,224.509995,225.990999,254.912922,226.672000,2022-11-09 00:00:00-05:00
2022-11-10 00:00:00-05:00,235.429993,243.330002,235.000000,242.979996,227.613998,254.810696,225.990999,2022-11-10 00:00:00-05:00


In [22]:
def find_crossover(fast_sma, prev_fast_sma, slow_sma):
    
    if fast_sma > slow_sma and prev_fast_sma < slow_sma:
        return 'bullish crossover'
    elif fast_sma < slow_sma and prev_fast_sma > slow_sma:
        return 'bearish crossover'
    
    return None


df['crossover'] = np.vectorize(find_crossover)(df['fast_sma'], df['prev_fast_sma'], df['slow_sma'])

signal = df[df['crossover'] == 'bullish crossover'].copy()
signal

,Open,High,Low,Close,fast_sma,slow_sma,prev_fast_sma,time,crossover
Date,,,,,,,,,
2018-11-13 00:00:00-05:00,102.880674,104.019005,102.010179,102.297157,103.426875,103.250100,103.119810,2018-11-13 00:00:00-05:00,bullish crossover
2018-11-15 00:00:00-05:00,100.865669,103.565289,99.828101,103.065712,103.468670,103.407496,103.294242,2018-11-15 00:00:00-05:00,bullish crossover
2018-11-16 00:00:00-05:00,102.873590,104.602876,102.604590,104.036057,103.717175,103.518381,103.468670,2018-11-16 00:00:00-05:00,bullish crossover
2018-12-10 00:00:00-05:00,100.683147,103.738227,99.808891,103.363541,104.527929,104.319229,104.093684,2018-12-10 00:00:00-05:00,bullish crossover
2019-02-19 00:00:00-05:00,103.555676,104.391502,103.546067,103.920746,102.453744,102.321622,102.220292,2019-02-19 00:00:00-05:00,bullish crossover
2020-04-14 00:00:00-04:00,165.211857,169.855386,164.234272,169.806503,158.090169,157.113604,156.773361,2020-04-14 00:00:00-04:00,bullish crossover
2020-11-05 00:00:00-05:00,218.193681,220.237652,217.319099,219.422028,206.152951,205.742613,205.327504,2020-11-05 00:00:00-05:00,bullish crossover
2022-08-05 00:00:00-04:00,278.558715,283.049183,278.089709,282.310760,273.267958,273.209834,271.017732,2022-08-05 00:00:00-04:00,bullish crossover


In [32]:
# visualize close price
fig = px.line(df, x=df.index, y=['Close', 'fast_sma', 'slow_sma'],  width=1000, height=400)

for i, row in signal.iterrows():

    fig.add_vline(x=row.time)
    
fig.show()

In [37]:
# creating backtest and position classes

class Position:
    def __init__(self, open_datetime, open_price, order_type, volume, sl, tp):
        self.open_datetime = open_datetime
        self.open_price = open_price
        self.order_type = order_type
        self.volume = volume
        self.sl = sl
        self.tp = tp
        self.close_datetime = None
        self.close_price = None
        self.profit = None
        self.status = 'Open'
        
    def close_position(self, close_datetime, close_price):
        self.close_datetime = close_datetime
        self.close_price = close_price
        self.profit = (self.close_price - self.open_price) * self.volume if self.order_type == 'buy' \
                                                                        else (self.open_price - self.close_price) * self.volume
        self.status = 'closed'
        
    def _asdict(self):
        return {
            'open_datetime': self.open_datetime,
            'open_price': self.open_price,
            'order_type': self.order_type,
            'volume': self.volume,
            'sl': self.sl,
            'tp': self.tp,
            'close_datetime': self.close_datetime,
            'close_price': self.close_price,
            'profit': self.profit,
            'status': self.status,
        }
        
        
class Strategy:
    def __init__(self, df, starting_balance, volume):
        self.starting_balance = starting_balance
        self.volume = volume
        self.positions = []
        self.data = df
        
    def get_positions_df(self):
        df = pd.DataFrame([position._asdict() for position in self.positions])
        df['pnl'] = df['profit'].cumsum() + self.starting_balance
        return df
        
    def add_position(self, position):
        self.positions.append(position)
        
        return True
        
# logic
    def run(self):
        for i, data in self.data.iterrows():
            
            if data.crossover == 'bearish crossover':
                for position in self.positions:
                    if position.status == 'Open':
                        position.close_position(data.time, data.Close)
            
            if data.crossover == 'bullish crossover':
                self.add_position(Position(data.time, data.Close, 'buy', self.volume, 0, 0))
        
        return self.get_positions_df()

In [38]:
sma_crossover_strategy = Strategy(df, 10000, 1)
result = sma_crossover_strategy.run()

result

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl
0,2018-11-13 00:00:00-05:00,102.297157,buy,1,0,0,2018-11-14 00:00:00-05:00,100.846466,-1.450691,closed,9998.549309
1,2018-11-15 00:00:00-05:00,103.065712,buy,1,0,0,2018-11-19 00:00:00-05:00,100.510208,-2.555504,closed,9995.993805
2,2018-11-16 00:00:00-05:00,104.036057,buy,1,0,0,2018-11-19 00:00:00-05:00,100.510208,-3.525848,closed,9992.467957
3,2018-12-10 00:00:00-05:00,103.363541,buy,1,0,0,2018-12-14 00:00:00-05:00,101.864830,-1.498711,closed,9990.969246
4,2019-02-19 00:00:00-05:00,103.920746,buy,1,0,0,2020-03-16 00:00:00-04:00,132.384567,28.463821,closed,10019.433067
5,2020-04-14 00:00:00-04:00,169.806503,buy,1,0,0,2020-11-03 00:00:00-05:00,202.854095,33.047592,closed,10052.480659
6,2020-11-05 00:00:00-05:00,219.422028,buy,1,0,0,2022-01-18 00:00:00-05:00,300.685425,81.263397,closed,10133.744057
7,2022-08-05 00:00:00-04:00,282.310760,buy,1,0,0,2022-09-02 00:00:00-04:00,256.059998,-26.250763,closed,10107.493294


In [39]:
px.line(result, x='close_datetime', y='pnl')

In [40]:
# visualize close price
fig = px.line(df, x='time', y=['Close', 'fast_sma', 'slow_sma'])

for i, row in signal.iterrows():
    fig.add_vline(x=row.time)
    
for i, row in result[result['status'] == 'closed'].iterrows():
    
    if row.profit > 0:
        fig.add_shape(type="line",
            x0=row.open_datetime, y0=row.open_price, x1=row.close_datetime, y1=row.close_price,
            line=dict(color="Green",width=3)
                     )
                      
    elif row.profit < 0:
        fig.add_shape(type="line",
            x0=row.open_datetime, y0=row.open_price, x1=row.close_datetime, y1=row.close_price,
            line=dict(color="Red",width=3)
                      )

    
fig.show()